In [ ]:
import numpy as np
import os
import pandas as pd

from scipy.stats import shapiro, kruskal
from scipy.special import expit
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from skbio.stats.distance import permanova
from skbio.stats.distance import DistanceMatrix
from scipy.spatial.distance import pdist, squareform
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# connect R and pandas
pandas2ri.activate()

# load R vegan
vegan = importr("vegan")
robjects.r('library(rcompanion)')

In [ ]:
data = pd.read_excel('datum\map_data.xlsx')

In [ ]:
def param_permanova(data, proc_var, participants, valence, MB = True):
    '''
    args:
        proc_var: input variable;
        participants: HC/ND;
        valence: rew/pun
    '''

    if proc_var == 'craving induce':
        score = pd.concat((data['pre'], data['ing']))
        raw_group = np.tile(data['type'], 2)
        group = ['ND' if raw_group[t] == 2 else 'HC' for t in range(len(raw_group))]
        time = ['before']*len(data) + ['after']*len(data)
        subject = ([f'subject{i}' for i in range(1, len(data)+1)]) * 2
        print('here, group: HC/ND; time: before/after')
    
    elif proc_var == 'craving decrease':
        use_value = 1 if participants == 'HC' else 2
        filtered_data = data[data['type'] == use_value]
        score = pd.concat((filtered_data['ing'], filtered_data['post']))
        raw_group = np.tile(filtered_data['background'], 2)
        group = ['neutral' if raw_group[t] == 1 else 'aversive' for t in range(len(raw_group))]
        time = ['before']*len(filtered_data) + ['after']*len(filtered_data)
        subject = ([f'subject{i}' for i in range(1, len(filtered_data)+1)]) * 2
        print('here, group: background; time: before/after')   

    elif proc_var == 'emotion':
        score = (data['emotion'])
        raw_group = data['type']
        group = ['ND' if raw_group[t] == 2 else 'HC' for t in range(len(raw_group))]
        raw_time = data['background']
        time =  ['aversive' if raw_time[t] == 2 else 'neutral' for t in range(len(raw_time))]
        subject = ([f'subject{i}' for i in range(1, len(data)+1)])
        print('here, group: HC/ND; time: background')   

    elif proc_var == 'best choice rate':
        score = (data[f'best choice rate_{valence}'])
        raw_group = data['type']
        group = ['ND' if raw_group[t] == 2 else 'HC' for t in range(len(raw_group))]
        raw_time = data['background']
        time =  ['aversive' if raw_time[t] == 2 else 'neutral' for t in range(len(raw_time))]
        subject = ([f'subject{i}' for i in range(1, len(data)+1)])        
        print('here, group: HC/ND; time: background')   

    elif proc_var in ['w', 'alpha1', 'alpha2','beta1', 'beta2', 'p', 'lambda']:
        use_value = 1 if participants == 'HC' else 2
        filtered_data = data[data['type'] == use_value]
        if MB:
            score = (pd.concat((filtered_data[f'{proc_var}_rew'], filtered_data[f'{proc_var}_pun'])))
        else: 
            score = (pd.concat((1-(filtered_data[f'{proc_var}_rew']), (1 - filtered_data[f'{proc_var}_pun']))))

        raw_group = filtered_data['background']
        group = ['aversive' if raw_group.iloc[t] == 2 else 'neutral' for t in range(len(filtered_data))] * 2
        time = ['reward'] * len(filtered_data) + ['punishment'] * len(filtered_data)
        subject = [f'subject{i}' for i in range(1, len(filtered_data) + 1)] * 2
        print('here, group: background; time: valence')

    elif proc_var == 'group_w':
        if MB:
            score = (data[f'w_{valence}'])
        else: 
            score = (1 - data[f'w_{valence}'])
        raw_group = data['type']
        group = ['ND' if raw_group[t] == 2 else 'HC' for t in range(len(raw_group))]
        raw_time = data['background']
        time =  ['aversive' if raw_time[t] == 2 else 'neutral' for t in range(len(raw_time))]
        subject = ([f'subject{i}' for i in range(1, len(data)+1)])        
        print('here, group: HC/ND; time: background')     

    df = pd.DataFrame({'value': score, 'group': group, 'time': time, 'subject': subject})
    
    return df

In [ ]:
ur_name =['craving induce', 'craving decrease', 'emotion', 'best choice rate' ]

new_df = param_permanova(data, 'w', 'HC', '', False)

# deine R model
r_code = """
function(data) {
  result <- scheirerRayHare(value ~ group * time, data = data)
  return(result)
}
"""

# transform R into Python
r_srh = robjects.r(r_code)

r_data = pandas2ri.py2rpy(new_df)

# run R 
result = r_srh(r_data)

print(result)
result[3][1]


#### 3 way interaction group * valence * background

In [ ]:
value = pd.concat([data['w_rew'], data['w_pun']], axis = 0, ignore_index = True)
subject = ([f'subject{i}' for i in range(1, len(data)+1)]) * 2
valence = (['reward'] * len(data) + ['punishment'] * len(data))
raw_group = np.tile(data['type'], 2)
raw_background = np.tile(data['background'], 2)
group = ['ND' if raw_group[t] == 2 else 'HC' for t in range(len(raw_group))]
background =  ['aversive' if raw_background[t] == 2 else 'neutral' for t in range(len(raw_background))]
df2 = pd.DataFrame({'value':value, 'group':group, 'background':background,'valence':valence, 'subject':subject})
